In [ ]:
import pandas as pd
pd.set_option("display.max_columns", 30)
import requests

from datetime import datetime
from dateutil.relativedelta import relativedelta

Taxi trips tranformation

In [340]:
current_datetime = datetime.now() - relativedelta(months=2)

formatted_datetime = current_datetime.strftime('%Y-%m-%d')

In [341]:
url = f"https://data.cityofchicago.org/resource/ajtu-isnz.json?$where=trip_start_timestamp >= '{formatted_datetime}T00:00:00' AND trip_start_timestamp <= '{formatted_datetime}T23:59:59'&$limit=30000"

response = requests.get(url)
data = response.json()


In [342]:
taxi_trips = pd.DataFrame(data)

In [343]:
taxi_trips.drop(['pickup_census_tract', 'dropoff_census_tract'], axis=1, inplace=True)
taxi_trips.drop(['pickup_centroid_location', 'dropoff_centroid_location'], axis=1, inplace=True)

taxi_trips.rename(columns={'pickup_community_area': 'pickup_community_area_id', 'dropoff_community_area': 'dropoff_community_area_id'}, inplace=True)
taxi_trips.dropna(inplace=True)

In [344]:
taxi_trips['datetime_for_weather'] = pd.to_datetime(taxi_trips['trip_start_timestamp']).dt.floor('h')
#taxi_trips['datetime_for_weather'] = taxi_trips['datetime_for_weather'].dt.floor('h')

In [345]:
taxi_trips.sample()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather
6892,65dfacc7e883bc70092f84dd20899fbbb4bafab8,324bed23f497ca6853a133b2d1aa48e56dcd151565a283...,2025-01-31T15:15:00.000,2025-01-31T16:00:00.000,3078,14.81,76,7,39.25,10.94,0,4,54.69,Credit Card,Taxicab Insurance Agency Llc,41.980264315,-87.913624596,41.922686284,-87.649488729,2025-01-31 15:00:00


Taxi trips transformation function

In [ ]:
def taxi_trips_transformation(taxi_trips: pd.DataFrame) -> pd.DataFrame:
    ''' 
    Transforms the taxi trips to a DataFrame

    Parameters:
        taxi_trips (pd.DataFrame): A DataFrame containing taxi trip data

    Returns:
        pd.DataFrame: The transformed DataFrame with the specified columns removed, renamed, 
                      and a new 'datetime_for_weather' column added.
    '''
    if not isinstance(taxi_trips, pd.DataFrame):
        raise ('taxi_trips is not a valid pandas DataFrame')

    taxi_trips.drop(['pickup_census_tract', 'dropoff_census_tract', 'pickup_centroid_location', 'dropoff_centroid_location'], axis=1, inplace=True)
    
    taxi_trips.dropna(inplace=True)
    
    taxi_trips.rename(columns={'pickup_community_area': 'pickup_community_area_id', 'dropoff_community_area': 'dropoff_community_area_id'}, inplace=True)
    
    taxi_trips['datetime_for_weather'] = pd.to_datetime(taxi_trips['trip_start_timestamp']).dt.floor('h')
    
    return taxi_trips

Company update code

In [348]:
company_master = taxi_trips['company'].drop_duplicates().reset_index(drop=True)

company_master = pd.DataFrame( 
    {
        'company_id': range(1, len(company_master) +1),
        'company': company_master
    }
)
company_master.tail()

,company_id,company
30,31,5167 - 71969 5167 Taxi Inc
31,32,U Taxicab
32,33,3556 - 36214 RC Andrews Cab
33,34,Tac - Yellow Non Color
34,35,Metro Jet Taxi A.


In [349]:
new_company = [
    {'company': 'Top Cab'},
    {'company': 'x'},
    {'company': 'y'}
]

new_company_mapping = pd.DataFrame(new_company)
new_company_mapping

,company
0,Top Cab
1,x
2,y


In [350]:
company_max_id = company_master['company_id'].max()

In [351]:
new_company_list = [company for company in new_company_mapping['company'].values if company not in company_master['company'].values]
new_company_list

['x', 'y']

In [352]:
new_companies_df = pd.DataFrame({
    "company_id": range(company_max_id +1, company_max_id + len(new_company_list)+1),
    "company": new_company_list
})

new_companies_df

,company_id,company
0,36,x
1,37,y


In [353]:
new_companies_df = pd.DataFrame(
    {
        "company_id": range(company_max_id +1, company_max_id + len(new_company_list)+1),
        "company": new_company_list
    }
)


In [354]:

new_companies_df

,company_id,company
0,36,x
1,37,y


In [355]:
updated_company_master = pd.concat([company_master, new_companies_df], ignore_index=True)

In [356]:
updated_company_master.tail()

,company_id,company
32,33,3556 - 36214 RC Andrews Cab
33,34,Tac - Yellow Non Color
34,35,Metro Jet Taxi A.
35,36,x
36,37,y


In [357]:
taxi_trips_company_only = pd.DataFrame({
    'company_id': [1, 2, 3],
    'company': ['CMetro Jet Taxi A.', 'x', 'y']
})

Company update function

In [ ]:
def update_company_master(taxi_trips: pd.DataFrame, company_master: pd.DataFrame) -> pd.DataFrame:
    """
    Extends the company master DataFrame by adding new companies found in the taxi trips data.

    Parameters
    ----------
    taxi_trips : pd.DataFrame
        DataFrame containing daily taxi trip records.
    
    company_master : pd.DataFrame
        DataFrame containing existing company mappings, with 'company_id' and 'company' columns.

    Returns
    -------
    pd.DataFrame
        The updated company master DataFrame. Any new unique company names found in the taxi trip data 
        that are not already in the master will be appended.

    """

    company_max_id = company_master["company_id"].max()

    new_companies_list = [company for company in taxi_trips["company"].values if company not in company_master["company"].values]
    new_companies_df = pd.DataFrame({
        "company_id": range(company_max_id +1, company_max_id + len(new_companies_list)+1),
        "company": new_companies_list
    })

    
    updated_company_master = pd.concat([company_master, new_companies_df], ignore_index=True)

    return updated_company_master

In [359]:
test_1= update_company_master(taxi_trips=taxi_trips_company_only, company_master=company_master)

In [360]:
test_1.tail()

,company_id,company
33,34,Tac - Yellow Non Color
34,35,Metro Jet Taxi A.
35,36,CMetro Jet Taxi A.
36,37,x
37,38,y


Payment type update code

In [ ]:
payment_type_master = taxi_trips['payment_type'].drop_duplicates().reset_index(drop=True)

payment_type_master = pd.DataFrame( 
    {
    'payment_type_id': range(1, len(payment_type_master) +1),
    'payment_type': payment_type_master

})
taxi_trips_payment_type_only = pd.DataFrame({
    'payment_type_id': [1, 2, 3],
    'payment_type': ['Credit Card', 'x', 'y']
})

In [361]:
payment_type_master = taxi_trips['payment_type'].drop_duplicates().reset_index(drop=True)

payment_type_master = pd.DataFrame( 
    {
        'payment_type_id': range(1, len(payment_type_master) +1),
        'payment_type': payment_type_master
    }
)

In [362]:
payment_type_master

,payment_type_id,payment_type
0,1,Credit Card
1,2,Mobile
2,3,Cash
3,4,Prcard
4,5,Unknown
5,6,No Charge
6,7,Dispute


In [363]:
new_payment_type = [
    {'payment_type': 'Credit Card'},
    {'payment_type': 'x'},
    {'payment_type': 'y'}
]

new_payment_type_mapping = pd.DataFrame(new_payment_type)
new_payment_type_mapping


,payment_type
0,Credit Card
1,x
2,y


In [364]:
payment_type_max_id = payment_type_master['payment_type_id'].max()

In [365]:
#for payment_type in new_payment_type_mapping['payment_type'].values:
#    if payment_type not in payment_type_master['payment_type'].values:
#        print(payment_type)

new_payment_type_list = [payment_type for payment_type in new_payment_type_mapping['payment_type'].values if payment_type not in payment_type_master['payment_type'].values]

In [366]:
new_payment_type_list

['x', 'y']

In [367]:
 
new_payment_type_df = pd.DataFrame(
    {
        "payment_type_id": range(payment_type_max_id + 1, payment_type_max_id + len(new_payment_type_list) + 1),
        "payment_type": new_payment_type_list
    }
)

In [368]:
new_payment_type_df

,payment_type_id,payment_type
0,8,x
1,9,y


In [369]:
updated_payment_type_master = pd.concat([payment_type_master, new_payment_type_df], ignore_index=True)

In [370]:
updated_payment_type_master

,payment_type_id,payment_type
0,1,Credit Card
1,2,Mobile
2,3,Cash
3,4,Prcard
4,5,Unknown
5,6,No Charge
6,7,Dispute
7,8,x
8,9,y


In [371]:
taxi_trips_payment_type_only = pd.DataFrame({
    'payment_type_id': [1, 2, 3],
    'payment_type': ['Credit Card', 'x', 'y']
})

Payment type update function

In [ ]:

def update_payment_type_master(taxi_trips: pd.DataFrame, payment_type_master: pd.DataFrame) -> pd.DataFrame:
    """
    Extends the payment type master DataFrame by adding new payment types found in the taxi trips data.

    Parameters
    ----------
    taxi_trips : pd.DataFrame
        DataFrame containing daily taxi trip records, including a 'payment_type' column.
    payment_type_master : pd.DataFrame
        DataFrame containing existing payment type mappings, with 'payment_type_id' and 'payment_type' columns.

    Returns
    -------
    pd.DataFrame
        The updated payment type master DataFrame. Any new unique payment types found in the taxi trip data 
        that are not already in the master will be appended.

    """

    payment_type_max_id = payment_type_master["payment_type_id"].max()

    new_payment_types_list = [payment_type for payment_type in taxi_trips["payment_type"].values if payment_type not in payment_type_master["payment_type"].values]
    new_payment_type_df = pd.DataFrame({
        "payment_type_id": range(payment_type_max_id + 1, payment_type_max_id + len(new_payment_types_list) + 1),
        "payment_type": new_payment_types_list
    })
    
    updated_payment_type_master = pd.concat([payment_type_master, new_payment_type_df], ignore_index=True)

    return updated_payment_type_master


In [373]:
test_2 =update_payment_type_master(taxi_trips=taxi_trips_payment_type_only, payment_type_master=payment_type_master)
test_2

,payment_type_id,payment_type
0,1,Credit Card
1,2,Mobile
2,3,Cash
3,4,Prcard
4,5,Unknown
5,6,No Charge
6,7,Dispute
7,8,x
8,9,y


Generic update master table function

In [ ]:
def update_master(taxi_trips: pd.DataFrame, master: pd.DataFrame, id_column: str, value_column: str ) -> pd.DataFrame:
    """
    Extends the master DataFrame by appending new unique values found in the taxi trips data.

    Parameters
    ----------
    taxi_trips : pd.DataFrame
        DataFrame containing daily taxi trip records.
    master : pd.DataFrame
        Master DataFrame containing existing mappings of IDs to values.
    id_column : str
        Name of the column in the master DataFrame that holds the ID.
    value_column : str
        Name of the column in both DataFrames that holds the value.

    Returns
    -------
    pd.DataFrame
        The updated master DataFrame.

    """

    max_id = master[id_column].max()

    new_value_list = [value for value in taxi_trips[value_column].values if value not in master[value_column].values]
    new_value_df = pd.DataFrame({
        id_column: range(max_id + 1, max_id + len(new_value_list) + 1),
        value_column: new_value_list
    })
    
    updated_master = pd.concat([master, new_value_df], ignore_index=True)

    return updated_master

In [375]:
test_3 = update_master(taxi_trips=taxi_trips_company_only, master=company_master, id_column='company_id', value_column='company' )

In [376]:
test_3

,company_id,company
0,1,Taxi Affiliation Services
1,2,Top Cab
2,3,Sun Taxi
3,4,Globe Taxi
4,5,Tac - Yellow Cab Association
5,6,City Service
6,7,5 Star Taxi
7,8,Flash Cab
8,9,Taxicab Insurance Agency Llc
9,10,Chicago Independents


update taxi trips with the most recenet company and payment type master

In [377]:
taxi_trips.head()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather
0,36caf85ede1a6f7e6f8f10b3027e3181e68983f1,b41e8c8124d8d600a4db8b7c176c2956756d378c749cb1...,2025-01-31T23:45:00.000,2025-01-31T23:45:00.000,480,1.1,28,8,8.32,0,0,0,8.32,Credit Card,Taxi Affiliation Services,41.88528132,-87.6572332,41.892042136,-87.63186395,2025-01-31 23:00:00
1,34b535748a8ba04dde92ab87a2c60d31a4c8fadd,5ffab897d6aec7c42dbe4fb2ce017ad47ea52ded99ebbf...,2025-01-31T23:45:00.000,2025-02-01T00:00:00.000,780,2.88,8,7,10.32,2.95,0,0,13.77,Mobile,Top Cab,41.899602111,-87.633308037,41.922686284,-87.649488729,2025-01-31 23:00:00
2,32b4602b7f47ff5b9e1f20ddb2ef95fccea27e35,2a342ab47780aec1f79d04cc913ac20c099335d24571cd...,2025-01-31T23:45:00.000,2025-02-01T00:00:00.000,719,3.73,28,33,11.4,0,0,0,11.9,Mobile,Sun Taxi,41.874005383,-87.66351755,41.857183858,-87.620334624,2025-01-31 23:00:00
3,2edd0d82e09e03fd3ee324c8b8b54868e7e8591c,d511072131b602026bdb9faa5491d15c3af8d62dc00659...,2025-01-31T23:45:00.000,2025-02-01T00:00:00.000,720,3.8,21,24,13.25,0,0,0,13.25,Cash,Taxi Affiliation Services,41.938666196,-87.711210593,41.901206994,-87.676355989,2025-01-31 23:00:00
4,2b2b157a4f7f8a99c392b933d727345f1f200832,397fda5f3737a7b1a8ee587706ebb7e2ba198216728b26...,2025-01-31T23:45:00.000,2025-01-31T23:45:00.000,606,2.06,28,8,9.5,2.92,0,0,12.92,Mobile,Globe Taxi,41.88528132,-87.6572332,41.890922026,-87.618868355,2025-01-31 23:00:00


update taxi_trips with company_master and payment_type master function

In [ ]:
def update_taxi_trips_with_master_data(taxi_trips: pd.DataFrame, payment_type_master: pd.DataFrame, company_master: pd.DataFrame) -> pd.DataFrame:
    """
    Update the taxi trips DataFrame with information from master data tables.

    Parameters
    ----------
    taxi_trips : pd.DataFrame
        A DataFrame containing taxi trip records, including 'payment_type' and 'company' columns.
    payment_type_master : pd.DataFrame
        Payment type master table.
    company_master : pd.DataFrame
        Company master table.

    Returns
    -------
    pd.DataFrame
        A DataFrame with the original trip data enriched with information from 
        the payment type and company master data.

    """
    taxi_trips_id = taxi_trips.merge(payment_type_master, on= 'payment_type')
    taxi_trips_id = taxi_trips_id.merge(company_master, on= 'company')
    taxi_trips_id.drop(["payment_type", "company"], axis=1, inplace=True)
    return taxi_trips_id

In [383]:
taxi_trips_id = update_taxi_trips_with_master_data(taxi_trips=taxi_trips, payment_type_master=payment_type_master, company_master=company_master)

In [385]:
taxi_trips_id.head()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather,payment_type_id,company_id
0,36caf85ede1a6f7e6f8f10b3027e3181e68983f1,b41e8c8124d8d600a4db8b7c176c2956756d378c749cb1...,2025-01-31T23:45:00.000,2025-01-31T23:45:00.000,480,1.1,28,8,8.32,0,0,0,8.32,41.88528132,-87.6572332,41.892042136,-87.63186395,2025-01-31 23:00:00,1,1
1,34b535748a8ba04dde92ab87a2c60d31a4c8fadd,5ffab897d6aec7c42dbe4fb2ce017ad47ea52ded99ebbf...,2025-01-31T23:45:00.000,2025-02-01T00:00:00.000,780,2.88,8,7,10.32,2.95,0,0,13.77,41.899602111,-87.633308037,41.922686284,-87.649488729,2025-01-31 23:00:00,2,2
2,32b4602b7f47ff5b9e1f20ddb2ef95fccea27e35,2a342ab47780aec1f79d04cc913ac20c099335d24571cd...,2025-01-31T23:45:00.000,2025-02-01T00:00:00.000,719,3.73,28,33,11.4,0,0,0,11.9,41.874005383,-87.66351755,41.857183858,-87.620334624,2025-01-31 23:00:00,2,3
3,2edd0d82e09e03fd3ee324c8b8b54868e7e8591c,d511072131b602026bdb9faa5491d15c3af8d62dc00659...,2025-01-31T23:45:00.000,2025-02-01T00:00:00.000,720,3.8,21,24,13.25,0,0,0,13.25,41.938666196,-87.711210593,41.901206994,-87.676355989,2025-01-31 23:00:00,3,1
4,2b2b157a4f7f8a99c392b933d727345f1f200832,397fda5f3737a7b1a8ee587706ebb7e2ba198216728b26...,2025-01-31T23:45:00.000,2025-01-31T23:45:00.000,606,2.06,28,8,9.5,2.92,0,0,12.92,41.88528132,-87.6572332,41.890922026,-87.618868355,2025-01-31 23:00:00,2,4


Weather transformation function

In [389]:
url = 'https://archive-api.open-meteo.com/v1/era5?'

current_datetime = datetime.now() - relativedelta(months=2)
formatted_datetime = current_datetime.strftime("%Y-%m-%d")
params = {
    "latitude": 41.85,
    "longitude": -87.65,
    "start_date": formatted_datetime,
    "end_date": formatted_datetime,
    "hourly": "temperature_2m,wind_speed_10m,rain,precipitation"
}


response = requests.get(url, params=params)
weather_data = response.json()

In [ ]:
def transform_weather_data(weather_data: json) -> pd.DataFrame:
    """
    Transforms raw weather data into a pandas DataFrame.

    Parameters
    ----------
    weather_data : dict
        A dictionary containing weather information with a structure that includes 
        'hourly' data for 'time', 'temperature_2m', 'wind_speed_10m', 'rain', and 'precipitation'.

    Returns
    -------
    pd.DataFrame
        A DataFrame with the following columns:
        - 'datetime': Converted to pandas datetime format
        - 'temperature': Temperature at 2 meters
        - 'wind_speed': Wind speed at 10 meters
        - 'rain': Rain amount
        - 'precipitation': Precipitation amount
    """
    
    weather_data_filtered = {
        'datetime': weather_data['hourly']["time"],
        'tempereature': weather_data['hourly']['temperature_2m'],
        'wind_speed': weather_data['hourly']['wind_speed_10m'],
        'rain': weather_data['hourly']['rain'],
        'precipitation': weather_data['hourly']['precipitation']
    }

    weather_df = pd.DataFrame(weather_data_filtered)
    weather_df['datetime'] = pd.to_datetime(weather_df['datetime'])

  
    return weather_df

In [392]:
test_weather =transform_weather_data(weather_data = weather_data)

In [393]:
test_weather

,datetime,tempereature,wind_speed,rain,precipitation
0,2025-01-31 00:00:00,5.6,12.7,2.7,2.7
1,2025-01-31 01:00:00,5.1,13.9,2.8,2.8
2,2025-01-31 02:00:00,5.0,12.4,9.1,9.1
3,2025-01-31 03:00:00,4.6,10.6,0.6,0.6
4,2025-01-31 04:00:00,4.4,9.4,0.7,0.7
5,2025-01-31 05:00:00,4.3,8.4,0.9,0.9
6,2025-01-31 06:00:00,4.2,3.4,0.5,0.5
7,2025-01-31 07:00:00,4.0,6.7,0.1,0.1
8,2025-01-31 08:00:00,3.6,6.1,0.4,0.4
9,2025-01-31 09:00:00,3.1,8.9,0.1,0.1
